In [13]:
!python -m pip install newsapi-python

In [39]:
API_KEY = "5b4c706f8faa432e8be611b071660a48"

**RETRIEVING THE NEWS FEED**

Get the news feed from news api using the API Calls.

In [46]:
import requests
import random
import spacy
from collections import defaultdict

# Loading spacy NLP model for keyword extraction
nlp = spacy.load("en_core_web_sm")


API_KEY = "5b4c706f8faa432e8be611b071660a48"
URL = "https://newsapi.org/v2/everything"

#  personas and their specific interests
user_personas = {
    "Alex Parker": ["AI", "cybersecurity", "blockchain", "startups", "programming"],
    "Priya Sharma": ["global markets", "startups", "fintech", "cryptocurrency", "economics"],
    "Marco Rossi": ["football", "F1", "NBA", "Olympic sports", "esports"],
    "Lisa Thompson": ["movies", "celebrity news", "TV shows", "music", "books"],
    "David Martinez": ["space exploration", "AI", "biotech", "physics", "renewable energy"]
}

# Fetching 100 random news articles without category restrictions
def fetch_news():
    params = {
        "q": "*",  # Wildcard to fetch diverse news
        "language": "en",
        "apiKey": API_KEY,
        "pageSize": 100,
        "sortBy": "publishedAt"
    }
    response = requests.get(URL, params=params)
    return response.json().get("articles", []) if response.status_code == 200 else []

# Extraction of keywords from text using NLP
def extract_keywords(text):
    doc = nlp(text.lower())
    return {token.lemma_ for token in doc if token.is_alpha and not token.is_stop}

# Assigning news articles to user personas
def assign_news_to_users():
    user_news = defaultdict(list)
    articles = fetch_news()
    random.shuffle(articles)  # Shuffle to diversify assignments

    for article in articles:
        title = article.get("title", "")
        description = article.get("description", "")
        text = f"{title} {description}"
        article_keywords = extract_keywords(text)

        # Score each article for each persona
        for user, interests in user_personas.items():
            interest_keywords = set(map(str.lower, interests))
            match_score = len(article_keywords & interest_keywords)

            if match_score > 0:
                user_news[user].append((match_score, {
                    "title": title,
                    "description": description,
                    "url": article.get("url", ""),
                    "image_url": article.get("urlToImage", ""),
                    "source": article.get("source", {}).get("name", "Unknown Source"),
                    "published_at": article.get("publishedAt", "")
                }))

    # Sort articles by relevance and ensure at least 5 per user
    final_user_news = {}
    for user in user_news:
        sorted_articles = [article for _, article in sorted(user_news[user], key=lambda x: x[0], reverse=True)]
        final_user_news[user] = sorted_articles[:max(5, len(sorted_articles))]  # Ensure at least 5 if possible

    return final_user_news

# Run the AI model to curate news
news_for_users = assign_news_to_users()

# Print curated news for each user
for user, articles in news_for_users.items():
    print(f"\nNews for {user}:")
    for article in articles:
        print(f"- Title: {article['title']}\n Summary: {article['description']}\n Source: {article['source']})\n  Url: {article['url']}\n")



News for Alex Parker:
- Title: At Adobe Summit, has agentic AI for marketers made its public debut?
 Summary: While half of the Vegas megaconference grapples with implementing current generative offerings, the other put a flag in the sand for AI’s much-vaunted next phase.Las Vegas, a neon beacon in the desert, has always been a symbol of humanity’s (putative) triumph…
 Source: The Drum)
  Url: https://www.thedrum.com/news/2025/03/21/adobe-summit-has-agentic-ai-marketers-made-its-public-debut

- Title: Limitless Pendant wearable AI device answers your questions and preserves your convos
 Summary: Meet the Limitless Pendant! This wearable AI device delivers personalized productivity features powered by your experiences and actions.   –Ace your Meetings: No more scrambling to catch up before a call! With the Limitless Pendant, all your online and real-w…
 Source: Gadget Flow)
  Url: https://thegadgetflow.com/product/limitless-pendant-wearable-ai-device/

- Title: The FX Revolution Is Und